<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/pytorch/20_probability/25_regression_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Example : Linear Regression using PyTorch<br>사례 : PyTorch를 이용한 선형회귀

PyTorch is a library specialized in (computational) machine learning originally developed by Meta AI<br>PyTorch 는 Meta AI에서 기원한 (전산) 기계 학습에 특화된 라이브러리


In [ ]:
import torch



In [ ]:
import pylab as py
import numpy.random as nr


nr.seed()


a = 0.5
b = 2.0

x_array = py.linspace(0, 5, 20 + 1)
y_true = a * x_array + b

w_array = nr.normal(0, 0.25, size=x_array.shape)
y_measurement = y_true + w_array



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

